<a href="https://colab.research.google.com/github/RRG314/RRG-AI/blob/steven%2Fplugin-system/rdt_lm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
RDT LANGUAGE MODEL + Topological Adam Optimizer Integration
Author: Steven Reid
License: Apache 2.0
"""

import torch
import torch.nn as nn
import numpy as np
import math
import random
from collections import defaultdict, Counter
from topological_adam import TopologicalAdam   # ← installed package

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Using: {device}")

# ═══════════════════════════════════════════════════════════════════
# RDT MATH
# ═══════════════════════════════════════════════════════════════════
def rdt_depth(n, alpha=1.5):
    """Doubly-logarithmic depth calculation"""
    if n < 2:
        return 0
    x, depth = n, 0
    while x > 1 and depth < 1000:
        d = max(2, int(math.log(x) ** alpha))
        if x < d:
            break
        x //= d
        depth += 1
    return depth


def generate_corpus():
    patterns = [
        "the man walked to the house",
        "the woman went to the market",
        "the child ran to the river",
        "he was happy and she was sad",
        "they found the truth and left quickly",
        "in the morning he woke up early",
        "at night she went to sleep",
        "the old king ruled the land wisely",
        "the young prince sought great adventure",
        "once there was a wise teacher",
        "the forest was dark and very quiet",
        "he said yes but she said no",
        "they walked together in peaceful silence",
        "the door opened and he entered slowly",
        "she spoke softly to the young child",
    ]
    corpus = []
    for _ in range(2000):
        for p in patterns:
            corpus.append(p.split())
    random.shuffle(corpus)
    vocab = set(w for s in corpus for w in s)
    return corpus, vocab


# ═══════════════════════════════════════════════════════════════════
# RDT LANGUAGE MODEL
# ═══════════════════════════════════════════════════════════════════
class RDTLanguageModel(nn.Module):
    def __init__(self, vocab, embedding_dim=128, alpha=1.5):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.alpha = alpha

        # Vocabulary
        self.word2idx = {"<PAD>": 0, "<UNK>": 1}
        for word in vocab:
            self.word2idx[word] = len(self.word2idx)
        self.idx2word = {v: k for k, v in self.word2idx.items()}
        self.vocab_size = len(self.word2idx)

        # ═══════════════════════════════════════════════════════════
        # Shell organization
        # ═══════════════════════════════════════════════════════════
        self.shells = defaultdict(list)
        self.word_depth = {}
        for word, idx in self.word2idx.items():
            if idx >= 2:
                depth = rdt_depth(idx, self.alpha)
                self.shells[depth].append(word)
                self.word_depth[word] = depth

        # Embeddings are now learnable Parameters
        self.embeddings = nn.ParameterDict(
            {
                w: nn.Parameter(
                    torch.randn(embedding_dim, device=device) * 0.1
                )
                for w in self.word2idx.keys()
            }
        )

        # n-gram structures (not learnable)
        self.bigrams = defaultdict(Counter)
        self.trigrams = defaultdict(Counter)

        print(f"\n✅ RDT Model Initialized:")
        print(f"   Vocabulary: {self.vocab_size} words")
        print(f"   Embedding dim: {embedding_dim}")
        print(f"   Alpha: {alpha}")
        for d in sorted(self.shells.keys()):
            print(f"      Shell {d}: {len(self.shells[d])} words")
            print(f"         Sample: {self.shells[d][:6]}")

    # ═══════════════════════════════════════════════════════════
    # Training with Topological Adam
    # ═══════════════════════════════════════════════════════════
    def train_rdt(self, corpus, epochs=10, lr=1e-3):
        print("\n🔥 Training with Topological Adam (shell-aware geometry)\n")
        optimizer = TopologicalAdam(self.embeddings.parameters(), lr=lr)
        criterion = nn.MSELoss()

        for epoch in range(epochs):
            random.shuffle(corpus)
            total_loss = 0.0
            for sentence in corpus:
                optimizer.zero_grad()

                # record n-grams (same as before)
                for i in range(len(sentence) - 1):
                    self.bigrams[sentence[i]][sentence[i + 1]] += 1
                if len(sentence) >= 3:
                    for i in range(len(sentence) - 2):
                        self.trigrams[(sentence[i], sentence[i + 1])][
                            sentence[i + 2]
                        ] += 1

                # build small loss from local word pairs
                losses = []
                for i, w in enumerate(sentence[:-1]):
                    w_next = sentence[i + 1]
                    if w in self.embeddings and w_next in self.embeddings:
                        e1, e2 = self.embeddings[w], self.embeddings[w_next]
                        sim = torch.dot(e1, e2) / (e1.norm() * e2.norm() + 1e-8)
                        losses.append((1 - sim) ** 2)

                if losses:
                    loss = torch.stack(losses).mean()
                    loss.backward()
                    optimizer.step()
                    total_loss += loss.item()

            print(
                f"Epoch {epoch+1}/{epochs}: "
                f"loss={total_loss:.6f}, energy={optimizer.energy():.6e}"
            )

        print("\n✅ Training Complete using Topological Adam!")

    # ═══════════════════════════════════════════════════════════
    # Generation (unchanged)
    # ═══════════════════════════════════════════════════════════
    def generate(self, start_text, max_length=15):
        words = start_text.lower().split()
        generated = list(words)
        recent = set()

        for _ in range(max_length):
            last = generated[-1]
            if last not in self.word2idx:
                break

            # grammar first
            if len(generated) >= 2:
                key = (generated[-2], generated[-1])
                if key in self.trigrams and self.trigrams[key]:
                    cand = {w: c for w, c in self.trigrams[key].items() if w not in recent}
                    if cand:
                        nxt = random.choices(list(cand.keys()), weights=list(cand.values()))[0]
                        generated.append(nxt)
                        recent.add(nxt)
                        if len(recent) > 5:
                            recent.pop()
                        continue

            scores = {}
            if last in self.bigrams:
                total = sum(self.bigrams[last].values())
                for w, c in self.bigrams[last].items():
                    penalty = 0.1 if w in recent else 1.0
                    scores[w] = (c / total) * 0.6 * penalty

            last_shell = self.word_depth.get(last, 0)
            last_emb = self.embeddings[last]
            for off in [0, 1, -1]:
                check = last_shell + off
                if check in self.shells:
                    for w in self.shells[check]:
                        if w == last or w in recent:
                            continue
                        e = self.embeddings[w]
                        sim = (
                            torch.dot(last_emb, e)
                            / (last_emb.norm() * e.norm() + 1e-8)
                        ).item()
                        if self.word_depth.get(w, 0) == last_shell:
                            sim *= 1.5
                        scores[w] = scores.get(w, 0) + max(0, sim) * 0.4

            if not scores:
                break
            sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
            top3 = sorted_words[:3]
            nxt = random.choice(top3)[0] if len(top3) > 1 else sorted_words[0][0]
            generated.append(nxt)
            recent.add(nxt)
            if len(recent) > 5:
                recent.pop()
            if len(generated) >= 4 and len(set(generated[-4:])) <= 2:
                break
        return " ".join(generated)


# ═══════════════════════════════════════════════════════════════════
# MAIN EXECUTION
# ═══════════════════════════════════════════════════════════════════
if __name__ == "__main__":
    print("=" * 70)
    print("RDT LANGUAGE MODEL WITH TOPOLOGICAL ADAM")
    print("=" * 70)

    torch.manual_seed(42)
    np.random.seed(42)
    random.seed(42)

    corpus, vocab = generate_corpus()
    print(f"\n📚 Training Data: {len(corpus):,} sentences, {len(vocab)} words")

    model = RDTLanguageModel(vocab, embedding_dim=128, alpha=1.5).to(device)
    model.train_rdt(corpus, epochs=10, lr=1e-3)

    prompts = ["the man walked", "she was happy", "in the morning", "the old king", "once there was"]
    print("\n" + "=" * 70)
    print("TEXT GENERATION")
    print("=" * 70)
    for p in prompts:
        print(f"\nPrompt: '{p}'")
        for i in range(3):
            print(f"  {i+1}. {model.generate(p, max_length=12)}")

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 116)

In [ ]:
!pip install topological-adam --upgrade


In [ ]:
"""
RDT LANGUAGE MODEL + Topological Adam Optimizer Integration
Author: Steven Reid
License: Apache 2.0
"""

import torch
import torch.nn as nn
import numpy as np
import math
import random
from collections import defaultdict, Counter
from topological_adam import TopologicalAdam  # pip install topological-adam

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Using: {device}")

# ═══════════════════════════════════════════════════════════════════
# RDT MATH
# ═══════════════════════════════════════════════════════════════════
def rdt_depth(n, alpha=1.5):
    """Doubly-logarithmic depth calculation"""
    if n < 2:
        return 0
    x, depth = n, 0
    while x > 1 and depth < 1000:
        d = max(2, int(math.log(x) ** alpha))
        if x < d:
            break
        x //= d
        depth += 1
    return depth


def generate_corpus():
    patterns = [
        "the man walked to the house",
        "the woman went to the market",
        "the child ran to the river",
        "he was happy and she was sad",
        "they found the truth and left quickly",
        "in the morning he woke up early",
        "at night she went to sleep",
        "the old king ruled the land wisely",
        "the young prince sought great adventure",
        "once there was a wise teacher",
        "the forest was dark and very quiet",
        "he said yes but she said no",
        "they walked together in peaceful silence",
        "the door opened and he entered slowly",
        "she spoke softly to the young child",
    ]
    corpus = []
    for _ in range(2000):
        for p in patterns:
            corpus.append(p.split())
    random.shuffle(corpus)
    vocab = set(w for s in corpus for w in s)
    return corpus, vocab


# ═══════════════════════════════════════════════════════════════════
# RDT LANGUAGE MODEL
# ═══════════════════════════════════════════════════════════════════
class RDTLanguageModel(nn.Module):
    def __init__(self, vocab, embedding_dim=128, alpha=1.5):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.alpha = alpha

        # Vocabulary
        self.word2idx = {"<PAD>": 0, "<UNK>": 1}
        for word in vocab:
            self.word2idx[word] = len(self.word2idx)
        self.idx2word = {v: k for k, v in self.word2idx.items()}
        self.vocab_size = len(self.word2idx)

        # Shell organization
        self.shells = defaultdict(list)
        self.word_depth = {}
        for word, idx in self.word2idx.items():
            if idx >= 2:
                depth = rdt_depth(idx, self.alpha)
                self.shells[depth].append(word)
                self.word_depth[word] = depth

        # Embeddings (safe keys to avoid collisions with nn.Module attrs)
        safe = {}
        for w in self.word2idx.keys():
            safe_key = f"w_{w}"
            safe[safe_key] = nn.Parameter(
                torch.randn(embedding_dim, device=device) * 0.1
            )
        self.embeddings = nn.ParameterDict(safe)
        self._embed_lookup = {
            w: self.embeddings[f"w_{w}"] for w in self.word2idx.keys()
        }

        # n-gram counters
        self.bigrams = defaultdict(Counter)
        self.trigrams = defaultdict(Counter)

        print(f"\n✅ RDT Model Initialized:")
        print(f"   Vocabulary: {self.vocab_size} words")
        print(f"   Embedding dim: {embedding_dim}")
        print(f"   Alpha: {alpha}")
        for d in sorted(self.shells.keys()):
            print(f"      Shell {d}: {len(self.shells[d])} words")
            print(f"         Sample: {self.shells[d][:6]}")

    # ═══════════════════════════════════════════════════════════
    # Training with Topological Adam
    # ═══════════════════════════════════════════════════════════
    def train_rdt(self, corpus, epochs=10, lr=1e-3):
        print("\n🔥 Training with Topological Adam (shell-aware geometry)\n")
        optimizer = TopologicalAdam(self.embeddings.parameters(), lr=lr)
        criterion = nn.MSELoss()

        for epoch in range(epochs):
            random.shuffle(corpus)
            total_loss = 0.0

            for sentence in corpus:
                optimizer.zero_grad()

                # record n-grams
                for i in range(len(sentence) - 1):
                    self.bigrams[sentence[i]][sentence[i + 1]] += 1
                if len(sentence) >= 3:
                    for i in range(len(sentence) - 2):
                        self.trigrams[(sentence[i], sentence[i + 1])][
                            sentence[i + 2]
                        ] += 1

                # compute loss from local word pairs
                losses = []
                for i, w in enumerate(sentence[:-1]):
                    w_next = sentence[i + 1]
                    if w in self._embed_lookup and w_next in self._embed_lookup:
                        e1 = self._embed_lookup[w]
                        e2 = self._embed_lookup[w_next]
                        sim = torch.dot(e1, e2) / (e1.norm() * e2.norm() + 1e-8)
                        losses.append((1 - sim) ** 2)

                if losses:
                    loss = torch.stack(losses).mean()
                    loss.backward()
                    optimizer.step()
                    total_loss += loss.item()

            print(
                f"Epoch {epoch+1}/{epochs}: "
                f"loss={total_loss:.6f}, energy={optimizer.energy():.6e}"
            )

        print("\n✅ Training Complete using Topological Adam!")

    # ═══════════════════════════════════════════════════════════
    # Generation (unchanged)
    # ═══════════════════════════════════════════════════════════
    def generate(self, start_text, max_length=15):
        words = start_text.lower().split()
        generated = list(words)
        recent = set()

        for _ in range(max_length):
            last = generated[-1]
            if last not in self.word2idx:
                break

            # grammar first
            if len(generated) >= 2:
                key = (generated[-2], generated[-1])
                if key in self.trigrams and self.trigrams[key]:
                    cand = {
                        w: c
                        for w, c in self.trigrams[key].items()
                        if w not in recent
                    }
                    if cand:
                        nxt = random.choices(
                            list(cand.keys()), weights=list(cand.values())
                        )[0]
                        generated.append(nxt)
                        recent.add(nxt)
                        if len(recent) > 5:
                            recent.pop()
                        continue

            scores = {}
            if last in self.bigrams:
                total = sum(self.bigrams[last].values())
                for w, c in self.bigrams[last].items():
                    penalty = 0.1 if w in recent else 1.0
                    scores[w] = (c / total) * 0.6 * penalty

            last_shell = self.word_depth.get(last, 0)
            last_emb = self._embed_lookup[last]
            for off in [0, 1, -1]:
                check = last_shell + off
                if check in self.shells:
                    for w in self.shells[check]:
                        if w == last or w in recent:
                            continue
                        e = self._embed_lookup[w]
                        sim = (
                            torch.dot(last_emb, e)
                            / (last_emb.norm() * e.norm() + 1e-8)
                        ).item()
                        if self.word_depth.get(w, 0) == last_shell:
                            sim *= 1.5
                        scores[w] = scores.get(w, 0) + max(0, sim) * 0.4

            if not scores:
                break
            sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
            top3 = sorted_words[:3]
            nxt = random.choice(top3)[0] if len(top3) > 1 else sorted_words[0][0]
            generated.append(nxt)
            recent.add(nxt)
            if len(recent) > 5:
                recent.pop()
            if len(generated) >= 4 and len(set(generated[-4:])) <= 2:
                break
        return " ".join(generated)


# ═══════════════════════════════════════════════════════════════════
# MAIN EXECUTION
# ═══════════════════════════════════════════════════════════════════
if __name__ == "__main__":
    print("=" * 70)
    print("RDT LANGUAGE MODEL WITH TOPOLOGICAL ADAM")
    print("=" * 70)

    torch.manual_seed(42)
    np.random.seed(42)
    random.seed(42)

    corpus, vocab = generate_corpus()
    print(f"\n📚 Training Data: {len(corpus):,} sentences, {len(vocab)} words")

    model = RDTLanguageModel(vocab, embedding_dim=128, alpha=1.5).to(device)
    model.train_rdt(corpus, epochs=10, lr=1e-3)

    prompts = [
        "the man walked",
        "she was happy",
        "in the morning",
        "the old king",
        "once there was",
    ]
    print("\n" + "=" * 70)
    print("TEXT GENERATION")
    print("=" * 70)
    for p in prompts:
        print(f"\nPrompt: '{p}'")
        for i in range(3):
            print(f"  {i+1}. {model.generate(p, max_length=12)}")

🚀 Using: cuda
RDT LANGUAGE MODEL WITH TOPOLOGICAL ADAM

📚 Training Data: 30,000 sentences, 62 words

✅ RDT Model Initialized:
   Vocabulary: 64 words
   Embedding dim: 128
   Alpha: 1.5
      Shell 1: 2 words
         Sample: ['they', 'together']
      Shell 2: 11 words
         Sample: ['woke', 'said', 'once', 'ran', 'woman', 'market']
      Shell 3: 49 words
         Sample: ['silence', 'entered', 'at', 'slowly', 'and', 'adventure']

🔥 Training with Topological Adam (shell-aware geometry)

Epoch 1/10: loss=706.582346, energy=6.722811e-03
Epoch 2/10: loss=1.867584, energy=6.722930e-03
Epoch 3/10: loss=0.216469, energy=6.722835e-03
Epoch 4/10: loss=0.032107, energy=5.762563e-03
Epoch 5/10: loss=0.005049, energy=4.802022e-03
Epoch 6/10: loss=0.000831, energy=5.762650e-03
Epoch 7/10: loss=0.000143, energy=4.802027e-03
Epoch 8/10: loss=0.000027, energy=5.762603e-03
Epoch 9/10: loss=0.000006, energy=6.723121e-03
Epoch 10/10: loss=0.000002, energy=4.802164e-03

✅ Training Complete using Top

In [ ]:
!pip install topological-adam --quiet

In [ ]:
# ===============================================================
#  🔬  RDT + Topological Adam + Neural Network Integration Test
# ===============================================================
import torch, torch.nn as nn, torch.nn.functional as F
import numpy as np, math, random
from collections import defaultdict, Counter
from topological_adam import TopologicalAdam     # your installed package

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Using: {device}")

# ===============================================================
# 1.  RDT MATH  (unchanged)
# ===============================================================
def rdt_depth(n, alpha=1.5):
    if n < 2: return 0
    x, depth = n, 0
    while x > 1 and depth < 1000:
        d = max(2, int(math.log(x) ** alpha))
        if x < d: break
        x //= d; depth += 1
    return depth

# ===============================================================
# 2.  DATA
# ===============================================================
def generate_corpus():
    patterns = [
        "the man walked to the house",
        "the woman went to the market",
        "the child ran to the river",
        "he was happy and she was sad",
        "they found the truth and left quickly",
        "in the morning he woke up early",
        "the old king ruled the land wisely",
        "once there was a wise teacher",
    ]
    corpus = [p.split() for p in patterns * 4000]
    random.shuffle(corpus)
    vocab = sorted(set(word for sent in corpus for word in sent))
    return corpus, vocab

corpus, vocab = generate_corpus()
print(f"📚 Training Data: {len(corpus):,} sentences, {len(vocab)} words")

# ===============================================================
# 3.  RDT LANGUAGE ENCODER + NEURAL NET HEAD
# ===============================================================
class RDTNet(nn.Module):
    """
    Combines:
      • RDT shell embeddings
      • Topological Adam optimizer physics
      • GRU neural network head
    """
    def __init__(self, vocab, emb_dim=128, hidden=256, alpha=1.5):
        super().__init__()
        self.alpha = alpha
        self.word2idx = {w:i for i,w in enumerate(vocab)}
        self.idx2word = {i:w for w,i in self.word2idx.items()}
        self.vocab_size = len(vocab)

        # --- RDT shell geometry ---
        depths = [rdt_depth(i+2, alpha) for i in range(self.vocab_size)]
        self.shell_index = torch.tensor(depths, device=device)
        self.embed = nn.Embedding(self.vocab_size, emb_dim)
        self.rnn = nn.GRU(emb_dim, hidden, batch_first=True)
        self.fc  = nn.Linear(hidden, self.vocab_size)

    def forward(self, x):
        e = self.embed(x)
        out, _ = self.rnn(e)
        return self.fc(out)

# ===============================================================
# 4.  TRAINING LOOP WITH TOPOLOGICAL ADAM
# ===============================================================
def encode_sentence(sent, word2idx, maxlen=8):
    idxs = [word2idx[w] for w in sent if w in word2idx]
    pad = [0]*(maxlen-len(idxs))
    return torch.tensor([idxs+pad], device=device)

def train_model(model, corpus, epochs=10, lr=1e-3):
    optimizer = TopologicalAdam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    loss_log, energy_log = [], []

    for ep in range(1, epochs+1):
        random.shuffle(corpus)
        total_loss = 0.0
        for sent in corpus:
            if len(sent) < 2: continue
            x = encode_sentence(sent[:-1], model.word2idx)
            y = encode_sentence(sent[1:], model.word2idx).view(-1)
            out = model(x).view(-1, model.vocab_size)
            loss = criterion(out, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        mean_loss = total_loss / len(corpus)
        mean_energy = optimizer.energy()
        loss_log.append(mean_loss)
        energy_log.append(mean_energy)
        print(f"Epoch {ep:2d}: loss={mean_loss:.6f}  energy={mean_energy:.6e}")

    return loss_log, energy_log

# ===============================================================
# 5.  RUN EXPERIMENT
# ===============================================================
model = RDTNet(vocab).to(device)
losses, energies = train_model(model, corpus, epochs=10, lr=1e-3)
print("✅ Training complete — RDT + GRU + Topological Adam.")

# ===============================================================
# 6.  PLOT RESULTS
# ===============================================================
import matplotlib.pyplot as plt
plt.figure(figsize=(9,4))
plt.subplot(1,2,1); plt.plot(losses); plt.title("Loss"); plt.xlabel("Epoch")
plt.subplot(1,2,2); plt.plot(energies); plt.title("Energy"); plt.xlabel("Epoch")
plt.tight_layout(); plt.show()

🚀 Using: cpu
📚 Training Data: 32,000 sentences, 37 words
Epoch  1: loss=0.093509  energy=6.722810e-03
Epoch  2: loss=0.089060  energy=6.722810e-03
Epoch  3: loss=0.088955  energy=6.722808e-03
Epoch  4: loss=0.089000  energy=6.722814e-03
Epoch  5: loss=0.088966  energy=6.722814e-03


KeyboardInterrupt: 

In [ ]:
# ===============================================================
#  COMPLETE RDT LANGUAGE MODEL
#  ALL COMPONENTS: RDT + TNN + Topological Adam + Spatial Index + MHD Insights
# ===============================================================
!pip install topological-adam --quiet

import torch, torch.nn as nn, torch.nn.functional as F
import numpy as np, math, random
from collections import defaultdict
from topological_adam import TopologicalAdam
from matplotlib import pyplot as plt
from numba import njit

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Device: {device}")

# ===============================================================
# 1. YOUR RDT MATH (unchanged)
# ===============================================================
def rdt_depth(n, alpha=1.5):
    if n < 2: return 0
    x, depth = n, 0
    while x > 1 and depth < 1000:
        d = max(2, int(math.log(x) ** alpha))
        if x < d: break
        x //= d
        depth += 1
    return depth

@njit(fastmath=True)
def rdt_grid_size(n, alpha=1.5):
    if n <= 1:
        return 2
    d = max(2, int(np.log(n + 1) ** alpha))
    return min(d, 32)

# ===============================================================
# 2. YOUR RDT SPATIAL INDEX (unchanged)
# ===============================================================
class RDTEmbeddingIndex:
    def __init__(self, vocab_size, embedding_dim, alpha=1.5):
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.alpha = alpha
        self.shells = defaultdict(list)
        self.word_depth = {}
        for idx in range(vocab_size):
            depth = rdt_depth(idx, alpha)
            self.shells[depth].append(idx)
            self.word_depth[idx] = depth
        self.shell_grid_sizes = {}
        for depth, words in self.shells.items():
            n = len(words)
            self.shell_grid_sizes[depth] = rdt_grid_size(n, alpha)
        print(f"\n🔬 RDT Spatial Index:")
        for depth in sorted(self.shells.keys()):
            n = len(self.shells[depth])
            g = self.shell_grid_sizes[depth]
            print(f"   Shell {depth}: {n} words, grid {g}×{g}")
    def find_nearest_in_shell(self, embedding, target_shell, embeddings_dict, k=5):
        if target_shell not in self.shells: return []
        candidates = self.shells[target_shell]
        distances = []
        for word_idx in candidates:
            emb = embeddings_dict[word_idx]
            sim = F.cosine_similarity(embedding.unsqueeze(0), emb.unsqueeze(0))
            distances.append((word_idx, sim.item()))
        distances.sort(key=lambda x: x[1], reverse=True)
        return [idx for idx, _ in distances[:k]]
    def hierarchical_search(self, embedding, current_word_idx, embeddings_dict, k=10):
        current_shell = self.word_depth.get(current_word_idx, 0)
        candidates = self.find_nearest_in_shell(embedding, current_shell, embeddings_dict, k=k)
        if len(candidates) < k:
            for offset in [1, -1, 2, -2]:
                nearby_shell = current_shell + offset
                if nearby_shell in self.shells:
                    more = self.find_nearest_in_shell(
                        embedding, nearby_shell, embeddings_dict, k=k-len(candidates)
                    )
                    candidates.extend(more)
                    if len(candidates) >= k: break
        return candidates[:k]

# ===============================================================
# 3. DATA (unchanged)
# ===============================================================
def generate_rich_corpus():
    subjects = ['the man','the woman','the child','the king','the queen','the soldier','the merchant','the teacher','the old man','the young woman','a stranger','the traveler','the farmer','the priest','the warrior','the beggar','the noble']
    verbs = ['walked','ran','went','came','spoke','thought','found','discovered','remembered','believed','understood','left','arrived','departed','searched','wondered','feared','hoped','carried','held','took','gave','brought','sent']
    locations = ['to the house','to the market','to the river','to the forest','to the castle','through the valley','across the land','to the mountain','to the sea','to the village','to the city','through the door','into the darkness','toward the light','beyond the horizon','near the gate']
    adjectives = ['happy','sad','brave','wise','afraid','calm','strong','gentle','proud','humble','kind','cruel','young','old','rich','poor','noble','simple','great','small']
    times = ['in the morning','at night','at dawn','at dusk','yesterday','long ago','one day','when winter came','when spring arrived','at midnight','before sunrise','after sunset']
    objects = ['a sword','a book','a letter','a key','a treasure','a secret','a truth','a dream','a memory','a promise','a gift']
    corpus = []
    for _ in range(4000):
        s,v,l = random.choice(subjects),random.choice(verbs),random.choice(locations)
        corpus.append(f"{s} {v} {l}".split())
    for _ in range(3000):
        s,adj = random.choice(subjects),random.choice(adjectives)
        corpus.append(f"{s} was {adj}".split())
    for _ in range(3000):
        t,s,v = random.choice(times),random.choice(subjects),random.choice(verbs)
        corpus.append(f"{t} {s} {v}".split())
    for _ in range(2000):
        s=random.choice(subjects); v=random.choice(['found','held','carried','brought','took']); o=random.choice(objects)
        corpus.append(f"{s} {v} {o}".split())
    for _ in range(2000):
        s1,adj1=random.choice(subjects),random.choice(adjectives)
        s2,adj2=random.choice(['he','she','they']),random.choice(adjectives)
        corpus.append(f"{s1} was {adj1} and {s2} was {adj2}".split())
    for _ in range(2000):
        s,adj=random.choice(['the','a']),random.choice(adjectives)
        noun=random.choice(['man','woman','child','king','soldier'])
        v=random.choice(verbs)
        corpus.append(f"{s} {adj} {noun} {v}".split())
    random.shuffle(corpus)
    vocab = sorted(set(word for sent in corpus for word in sent))
    print(f"📚 Corpus: {len(corpus):,} sentences, {len(vocab)} words")
    return corpus, vocab

corpus, vocab = generate_rich_corpus()

# ===============================================================
# 4. TOPOLOGICAL NEURAL NETWORK (patched E/C scaling fix)
# ===============================================================
class TopologicalNeuralNetwork(nn.Module):
    def __init__(self, input_dim=256, hidden_dim=512, output_dim=256, lambda_topo=1.0):
        super().__init__()
        self.lambda_topo = lambda_topo
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.layer_norm = nn.LayerNorm(hidden_dim)
        self.energy = self.coupling = self.stability = 0.0
        self.energy_ratio = 1.0

    def compute_topological_terms(self, activations):
        eps = 1e-8
        E = torch.mean(activations ** 2)
        grad = torch.autograd.grad(E, activations, retain_graph=True, create_graph=True, allow_unused=True)
        if grad[0] is None:
            C = E
        else:
            C_raw = torch.mean(grad[0] ** 2)
            N = activations.numel()
            scale = 4.0 / (N ** 2)
            C = C_raw / (scale + eps)
        ratio = E / (C + eps)
        ratio_clamped = torch.clamp(ratio, 0.25, 4.0)
        E_d, C_d, S_d = E.detach(), C.detach(), torch.abs(ratio - 1.0).detach()
        return E_d, C_d, S_d, ratio_clamped

    def forward(self, x):
        reshaped = False
        if len(x.shape) == 3:
            b, s, d = x.shape
            x = x.reshape(b * s, d)
            reshaped = True
        h1 = F.relu(self.fc1(x))
        h2 = F.relu(self.fc2(h1))
        h2_norm = self.layer_norm(h2)
        out = self.fc3(h2_norm)
        if self.training:
            E_d, C_d, S_d, ratio_for_ctrl = self.compute_topological_terms(h2_norm)
            self.energy, self.coupling, self.stability = E_d.item(), C_d.item(), S_d.item()
            if not hasattr(self, "_ratio_ema"):
                self._ratio_ema = ratio_for_ctrl.detach()
            self._ratio_ema = 0.9 * self._ratio_ema + 0.1 * ratio_for_ctrl.detach()
            correction = self.lambda_topo * torch.log(self._ratio_ema + 1e-8)
            out = out + correction * torch.tanh(out)
        if reshaped:
            out = out.reshape(b, s, -1)
        return out

# ===============================================================
# 5–9  REMAIN UNCHANGED (your full system, loss, optimizer, visualization)
# ===============================================================
# ... [keep all remaining code from your working version unchanged below this line]

KeyboardInterrupt: 

In [ ]:
# ===============================================================
#  COMPLETE RDT LANGUAGE MODEL
#  ALL COMPONENTS: RDT + TNN + Topological Adam + Spatial Index + MHD Insights
#  (includes corrected E/C normalization)
# ===============================================================
!pip install topological-adam --quiet

import torch, torch.nn as nn, torch.nn.functional as F
import numpy as np, math, random
from collections import defaultdict
from topological_adam import TopologicalAdam
from matplotlib import pyplot as plt
from numba import njit

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Device: {device}")

# ===============================================================
# 1. YOUR RDT MATH
# ===============================================================
def rdt_depth(n, alpha=1.5):
    if n < 2: return 0
    x, depth = n, 0
    while x > 1 and depth < 1000:
        d = max(2, int(math.log(x) ** alpha))
        if x < d: break
        x //= d
        depth += 1
    return depth

@njit(fastmath=True)
def rdt_grid_size(n, alpha=1.5):
    if n <= 1:
        return 2
    d = max(2, int(np.log(n + 1) ** alpha))
    return min(d, 32)

# ===============================================================
# 2. RDT SPATIAL INDEX
# ===============================================================
class RDTEmbeddingIndex:
    def __init__(self, vocab_size, embedding_dim, alpha=1.5):
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.alpha = alpha
        self.shells = defaultdict(list)
        self.word_depth = {}
        for idx in range(vocab_size):
            depth = rdt_depth(idx, alpha)
            self.shells[depth].append(idx)
            self.word_depth[idx] = depth
        self.shell_grid_sizes = {}
        for depth, words in self.shells.items():
            n = len(words)
            self.shell_grid_sizes[depth] = rdt_grid_size(n, alpha)
        print(f"\n🔬 RDT Spatial Index:")
        for depth in sorted(self.shells.keys()):
            n = len(self.shells[depth])
            g = self.shell_grid_sizes[depth]
            print(f"   Shell {depth}: {n} words, grid {g}×{g}")

    def find_nearest_in_shell(self, embedding, target_shell, embeddings_dict, k=5):
        if target_shell not in self.shells:
            return []
        candidates = self.shells[target_shell]
        distances = []
        for word_idx in candidates:
            emb = embeddings_dict[word_idx]
            sim = F.cosine_similarity(embedding.unsqueeze(0), emb.unsqueeze(0))
            distances.append((word_idx, sim.item()))
        distances.sort(key=lambda x: x[1], reverse=True)
        return [idx for idx, _ in distances[:k]]

    def hierarchical_search(self, embedding, current_word_idx, embeddings_dict, k=10):
        current_shell = self.word_depth.get(current_word_idx, 0)
        candidates = self.find_nearest_in_shell(embedding, current_shell, embeddings_dict, k=k)
        if len(candidates) < k:
            for offset in [1, -1, 2, -2]:
                nearby_shell = current_shell + offset
                if nearby_shell in self.shells:
                    more = self.find_nearest_in_shell(
                        embedding, nearby_shell, embeddings_dict, k=k-len(candidates)
                    )
                    candidates.extend(more)
                    if len(candidates) >= k:
                        break
        return candidates[:k]

# ===============================================================
# 3. DATA
# ===============================================================
def generate_rich_corpus():
    subjects = ['the man','the woman','the child','the king','the queen','the soldier','the merchant','the teacher','the old man','the young woman','a stranger','the traveler','the farmer','the priest','the warrior','the beggar','the noble']
    verbs = ['walked','ran','went','came','spoke','thought','found','discovered','remembered','believed','understood','left','arrived','departed','searched','wondered','feared','hoped','carried','held','took','gave','brought','sent']
    locations = ['to the house','to the market','to the river','to the forest','to the castle','through the valley','across the land','to the mountain','to the sea','to the village','to the city','through the door','into the darkness','toward the light','beyond the horizon','near the gate']
    adjectives = ['happy','sad','brave','wise','afraid','calm','strong','gentle','proud','humble','kind','cruel','young','old','rich','poor','noble','simple','great','small']
    times = ['in the morning','at night','at dawn','at dusk','yesterday','long ago','one day','when winter came','when spring arrived','at midnight','before sunrise','after sunset']
    objects = ['a sword','a book','a letter','a key','a treasure','a secret','a truth','a dream','a memory','a promise','a gift']
    corpus = []
    for _ in range(4000):
        s,v,l = random.choice(subjects),random.choice(verbs),random.choice(locations)
        corpus.append(f"{s} {v} {l}".split())
    for _ in range(3000):
        s,adj = random.choice(subjects),random.choice(adjectives)
        corpus.append(f"{s} was {adj}".split())
    for _ in range(3000):
        t,s,v = random.choice(times),random.choice(subjects),random.choice(verbs)
        corpus.append(f"{t} {s} {v}".split())
    for _ in range(2000):
        s=random.choice(subjects); v=random.choice(['found','held','carried','brought','took']); o=random.choice(objects)
        corpus.append(f"{s} {v} {o}".split())
    for _ in range(2000):
        s1,adj1=random.choice(subjects),random.choice(adjectives)
        s2,adj2=random.choice(['he','she','they']),random.choice(adjectives)
        corpus.append(f"{s1} was {adj1} and {s2} was {adj2}".split())
    for _ in range(2000):
        s,adj=random.choice(['the','a']),random.choice(adjectives)
        noun=random.choice(['man','woman','child','king','soldier'])
        v=random.choice(verbs)
        corpus.append(f"{s} {adj} {noun} {v}".split())
    random.shuffle(corpus)
    vocab = sorted(set(word for sent in corpus for word in sent))
    print(f"📚 Corpus: {len(corpus):,} sentences, {len(vocab)} words")
    return corpus, vocab

corpus, vocab = generate_rich_corpus()

# ===============================================================
# 4. TOPOLOGICAL NEURAL NETWORK (with E/C normalization fix)
# ===============================================================
class TopologicalNeuralNetwork(nn.Module):
    def __init__(self, input_dim=256, hidden_dim=512, output_dim=256, lambda_topo=1.0):
        super().__init__()
        self.lambda_topo = lambda_topo
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.layer_norm = nn.LayerNorm(hidden_dim)
        self.energy = self.coupling = self.stability = 0.0
        self.energy_ratio = 1.0

    def compute_topological_terms(self, activations):
        eps = 1e-8
        E = torch.mean(activations ** 2)
        grad = torch.autograd.grad(E, activations, retain_graph=True, create_graph=True, allow_unused=True)
        if grad[0] is None:
            C = E
        else:
            C_raw = torch.mean(grad[0] ** 2)
            N = activations.numel()
            scale = 4.0 / (N ** 2)
            C = C_raw / (scale + eps)
        ratio = E / (C + eps)
        ratio_clamped = torch.clamp(ratio, 0.25, 4.0)
        E_d, C_d, S_d = E.detach(), C.detach(), torch.abs(ratio - 1.0).detach()
        return E_d, C_d, S_d, ratio_clamped

    def forward(self, x):
        reshaped = False
        if len(x.shape) == 3:
            b, s, d = x.shape
            x = x.reshape(b * s, d)
            reshaped = True
        h1 = F.relu(self.fc1(x))
        h2 = F.relu(self.fc2(h1))
        h2_norm = self.layer_norm(h2)
        out = self.fc3(h2_norm)
        if self.training:
            E_d, C_d, S_d, ratio_for_ctrl = self.compute_topological_terms(h2_norm)
            self.energy, self.coupling, self.stability = E_d.item(), C_d.item(), S_d.item()
            if not hasattr(self, "_ratio_ema"):
                self._ratio_ema = ratio_for_ctrl.detach()
            self._ratio_ema = 0.9 * self._ratio_ema + 0.1 * ratio_for_ctrl.detach()
            correction = self.lambda_topo * torch.log(self._ratio_ema + 1e-8)
            out = out + correction * torch.tanh(out)
        if reshaped:
            out = out.reshape(b, s, -1)
        return out

# ===============================================================
# 5. COMPLETE RDT MODEL
# ===============================================================
class RDTLanguageModel(nn.Module):
    def __init__(self, vocab, emb_dim=256, hidden_dim=512, alpha=1.5):
        super().__init__()
        self.alpha = alpha
        self.word2idx = {w:i for i,w in enumerate(vocab)}
        self.idx2word = {i:w for w,i in self.word2idx.items()}
        self.vocab_size = len(vocab)
        self.rdt_index = RDTEmbeddingIndex(self.vocab_size, emb_dim, alpha)
        self.embed = nn.Embedding(self.vocab_size, emb_dim)
        self.tnn = TopologicalNeuralNetwork(
            input_dim=emb_dim, hidden_dim=hidden_dim, output_dim=emb_dim, lambda_topo=1.0)
        self.fc_out = nn.Linear(emb_dim, self.vocab_size)
        print(f"\n📊 Complete Model:")
        print(f"   Parameters: {sum(p.numel() for p in self.parameters()):,}")
        print(f"   Components: RDT Index + TNN (MHD) + Spatial Search")

    def forward(self, x):
        e = self.embed(x)
        h = self.tnn(e)
        out = self.fc_out(h)
        return out, h

    def compute_rdt_spatial_loss(self, predictions, targets, hidden_states):
        batch_size, seq_len, vocab_size = predictions.shape
        pred_words = torch.argmax(predictions, dim=-1).view(-1)
        target_words = targets.view(-1)
        with torch.no_grad():
            embeddings_dict = {idx: self.embed.weight[idx] for idx in range(self.vocab_size)}
        total_bonus, n_valid = 0.0, 0
        sample_size = min(100, len(pred_words))
        indices = random.sample(range(len(pred_words)), sample_size)
        for i in indices:
            pred_idx = pred_words[i].item()
            target_idx = target_words[i].item()
            if target_idx == 0: continue
            hidden = hidden_states.view(-1, hidden_states.size(-1))[i]
            pred_depth = self.rdt_index.word_depth[pred_idx]
            target_depth = self.rdt_index.word_depth[target_idx]
            if pred_depth == target_depth:
                depth_bonus = 1.0
            else:
                min_depth = min(pred_depth, target_depth) + 1
                max_depth = max(pred_depth, target_depth) + 1
                depth_ratio = min_depth / max_depth
                depth_bonus = depth_ratio
            candidates = self.rdt_index.hierarchical_search(hidden, target_idx, embeddings_dict, k=10)
            if pred_idx in candidates:
                rank = candidates.index(pred_idx)
                rank_bonus = 1.0 / (rank + 1)
                combined_bonus = depth_bonus * rank_bonus
                total_bonus += combined_bonus
            else:
                total_bonus += depth_bonus * 0.1
            n_valid += 1
        return total_bonus / n_valid if n_valid > 0 else 0.0

# ===============================================================
# 6. TRAINING
# ===============================================================
def prepare_batch(sentences, word2idx, maxlen=15):
    batch_x, batch_y = [], []
    for sent in sentences:
        if len(sent) < 2: continue
        idxs = [word2idx.get(w, 0) for w in sent]
        if len(idxs) < 2: continue
        x, y = idxs[:-1], idxs[1:]
        if len(x) < maxlen:
            x = x + [0]*(maxlen-len(x)); y = y + [0]*(maxlen-len(y))
        else:
            x, y = x[:maxlen], y[:maxlen]
        batch_x.append(x); batch_y.append(y)
    if not batch_x: return None, None
    return torch.tensor(batch_x, device=device), torch.tensor(batch_y, device=device)

def train_model(model, corpus, epochs=50, initial_lr=1e-3, batch_size=64):
    optimizer = TopologicalAdam(model.parameters(), lr=initial_lr)
    criterion = nn.CrossEntropyLoss()
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
    loss_log, spatial_log, energy_log, coupling_log, ratio_log, topo_energy_log = [], [], [], [], [], []
    print(f"\n🔥 Training COMPLETE SYSTEM WITH MHD INSIGHTS:")
    print(f"   ✓ RDT spatial index\n   ✓ Topological Neural Network (MHD)\n   ✓ Topological Adam optimizer\n   ✓ Hierarchical search\n   ✓ MHD-inspired loss function\n")
    best_loss = float('inf')
    for ep in range(1, epochs+1):
        random.shuffle(corpus)
        epoch_loss = epoch_spatial = epoch_tnn_energy = epoch_tnn_coupling = epoch_ratio = 0.0
        n_batches = 0
        for i in range(0, len(corpus), batch_size):
            batch = corpus[i:i+batch_size]
            x, y = prepare_batch(batch, model.word2idx)
            if x is None: continue
            out, hidden = model(x)
            loss = criterion(out.view(-1, model.vocab_size), y.view(-1))
            spatial_bonus = model.compute_rdt_spatial_loss(out, y, hidden)
            spatial_weight = 0.5
            total_loss = loss - (spatial_weight * spatial_bonus)
            optimizer.zero_grad()
            total_loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            epoch_loss += loss.item()
            epoch_spatial += spatial_bonus
            epoch_tnn_energy += model.tnn.energy
            epoch_tnn_coupling += model.tnn.coupling
            epoch_ratio += model.tnn.energy_ratio
            n_batches += 1
        if n_batches:
            mean_loss = epoch_loss / n_batches
            mean_spatial = epoch_spatial / n_batches
            mean_tnn_energy = epoch_tnn_energy / n_batches
            mean_tnn_coupling = epoch_tnn_coupling / n_batches
            mean_ratio = epoch_ratio / n_batches
            mean_topo_energy = optimizer.energy()
            loss_log.append(mean_loss); spatial_log.append(mean_spatial)
            energy_log.append(mean_tnn_energy); coupling_log.append(mean_tnn_coupling)
            ratio_log.append(mean_ratio); topo_energy_log.append(mean_topo_energy)
            scheduler.step(mean_loss)
            marker = "🔥" if mean_loss < best_loss else ""
            best_loss = min(best_loss, mean_loss)
            if ep % 5 == 0 or ep == 1:
                print(f"Epoch {ep:2d}: loss={mean_loss:.4f} | spatial={mean_spatial:.4f} | E/C_ratio={mean_ratio:.3f} | TNN_E={mean_tnn_energy:.2e} | Adam_E={mean_topo_energy:.2e} {marker}")
    print(f"\n✅ Training Complete! Best loss: {best_loss:.4f}")
    return loss_log, spatial_log, energy_log, coupling_log, ratio_log, topo_energy_log

# ===============================================================
# 7. TEXT GENERATION
# ===============================================================
def generate_text(model, start_text, max_len=15, temperature=0.8):
    model.eval(); words = start_text.lower().split()
    with torch.no_grad():
        for _ in range(max_len):
            x = torch.tensor([[model.word2idx.get(w, 0) for w in words]], device=device)
            out, _ = model(x)
            logits = out[0, -1, :] / temperature
            probs = F.softmax(logits, dim=0)
            next_idx = torch.multinomial(probs, 1).item()
            if next_idx == 0: break
            words.append(model.idx2word[next_idx])
            if len(words) > 20: break
    return ' '.join(words)

# ===============================================================
# 8. RUN COMPLETE SYSTEM
# ===============================================================
print("="*70)
print("COMPLETE SYSTEM: RDT + TNN (MHD) + TOPOLOGICAL ADAM + SPATIAL INDEX")
print("="*70)

torch.manual_seed(42)
model = RDTLanguageModel(vocab, emb_dim=256, hidden_dim=512, alpha=1.5).to(device)
losses, spatials, energies, couplings, ratios, adam_energies = train_model(model, corpus, epochs=50, initial_lr=1e-3, batch_size=64)

print("\n" + "="*70)
print("TEXT GENERATION")
print("="*70)
for prompt in ["the man

SyntaxError: unterminated string literal (detected at line 337) (ipython-input-3959432459.py, line 337)

In [ ]:
# ===============================================================
#  COMPLETE RDT LANGUAGE MODEL
#  ALL COMPONENTS: RDT + TNN + Topological Adam + Spatial Index + MHD Insights
#  (with corrected E/C normalization inside TNN)
# ===============================================================
!pip install topological-adam --quiet

import torch, torch.nn as nn, torch.nn.functional as F
import numpy as np, math, random
from collections import defaultdict
from topological_adam import TopologicalAdam
from matplotlib import pyplot as plt
from numba import njit

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Device: {device}")

# ===============================================================
# 1. YOUR RDT MATH (unchanged)
# ===============================================================
def rdt_depth(n, alpha=1.5):
    """YOUR RDT depth formula"""
    if n < 2: return 0
    x, depth = n, 0
    while x > 1 and depth < 1000:
        d = max(2, int(math.log(x) ** alpha))
        if x < d: break
        x //= d
        depth += 1
    return depth

@njit(fastmath=True)
def rdt_grid_size(n, alpha=1.5):
    """YOUR adaptive grid sizing from spatial index"""
    if n <= 1:
        return 2
    d = max(2, int(np.log(n + 1) ** alpha))
    return min(d, 32)

# ===============================================================
# 2. YOUR RDT SPATIAL INDEX (unchanged)
# ===============================================================
class RDTEmbeddingIndex:
    """
    YOUR spatial index structure adapted to embedding space
    Uses YOUR hierarchical search and adaptive grid sizing
    """
    def __init__(self, vocab_size, embedding_dim, alpha=1.5):
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.alpha = alpha

        # Build shell structure using YOUR formula
        self.shells = defaultdict(list)
        self.word_depth = {}
        for idx in range(vocab_size):
            depth = rdt_depth(idx, alpha)
            self.shells[depth].append(idx)
            self.word_depth[idx] = depth

        # YOUR adaptive grid sizes per shell
        self.shell_grid_sizes = {}
        for depth, words in self.shells.items():
            n = len(words)
            self.shell_grid_sizes[depth] = rdt_grid_size(n, alpha)

        print(f"\n🔬 RDT Spatial Index:")
        for depth in sorted(self.shells.keys()):
            n = len(self.shells[depth])
            g = self.shell_grid_sizes[depth]
            print(f"   Shell {depth}: {n} words, grid {g}×{g}")

    def find_nearest_in_shell(self, embedding, target_shell, embeddings_dict, k=5):
        """YOUR spatial search within a shell"""
        if target_shell not in self.shells:
            return []
        candidates = self.shells[target_shell]
        distances = []
        for word_idx in candidates:
            emb = embeddings_dict[word_idx]
            sim = F.cosine_similarity(embedding.unsqueeze(0), emb.unsqueeze(0))
            distances.append((word_idx, sim.item()))
        distances.sort(key=lambda x: x[1], reverse=True)
        return [idx for idx, _ in distances[:k]]

    def hierarchical_search(self, embedding, current_word_idx, embeddings_dict, k=10):
        """YOUR hierarchical traversal strategy"""
        current_shell = self.word_depth.get(current_word_idx, 0)

        # Search current shell first
        candidates = self.find_nearest_in_shell(
            embedding, current_shell, embeddings_dict, k=k
        )

        # Expand to nearby shells if needed (YOUR expansion strategy)
        if len(candidates) < k:
            for offset in [1, -1, 2, -2]:
                nearby_shell = current_shell + offset
                if nearby_shell in self.shells:
                    more = self.find_nearest_in_shell(
                        embedding, nearby_shell, embeddings_dict, k=k-len(candidates)
                    )
                    candidates.extend(more)
                    if len(candidates) >= k:
                        break
        return candidates[:k]

# ===============================================================
# 3. DATA (unchanged)
# ===============================================================
def generate_rich_corpus():
    subjects = [
        'the man', 'the woman', 'the child', 'the king', 'the queen',
        'the soldier', 'the merchant', 'the teacher', 'the old man',
        'the young woman', 'a stranger', 'the traveler', 'the farmer',
        'the priest', 'the warrior', 'the beggar', 'the noble'
    ]
    verbs = [
        'walked', 'ran', 'went', 'came', 'spoke', 'thought', 'found',
        'discovered', 'remembered', 'believed', 'understood', 'left',
        'arrived', 'departed', 'searched', 'wondered', 'feared', 'hoped',
        'carried', 'held', 'took', 'gave', 'brought', 'sent'
    ]
    locations = [
        'to the house', 'to the market', 'to the river', 'to the forest',
        'to the castle', 'through the valley', 'across the land',
        'to the mountain', 'to the sea', 'to the village', 'to the city',
        'through the door', 'into the darkness', 'toward the light',
        'beyond the horizon', 'near the gate'
    ]
    adjectives = [
        'happy', 'sad', 'brave', 'wise', 'afraid', 'calm', 'strong',
        'gentle', 'proud', 'humble', 'kind', 'cruel', 'young', 'old',
        'rich', 'poor', 'noble', 'simple', 'great', 'small'
    ]
    times = [
        'in the morning', 'at night', 'at dawn', 'at dusk', 'yesterday',
        'long ago', 'one day', 'when winter came', 'when spring arrived',
        'at midnight', 'before sunrise', 'after sunset'
    ]
    objects = [
        'a sword', 'a book', 'a letter', 'a key', 'a treasure', 'a secret',
        'a truth', 'a dream', 'a memory', 'a promise', 'a gift'
    ]

    corpus = []
    for _ in range(4000):
        s, v, l = random.choice(subjects), random.choice(verbs), random.choice(locations)
        corpus.append(f"{s} {v} {l}".split())
    for _ in range(3000):
        s, adj = random.choice(subjects), random.choice(adjectives)
        corpus.append(f"{s} was {adj}".split())
    for _ in range(3000):
        t, s, v = random.choice(times), random.choice(subjects), random.choice(verbs)
        corpus.append(f"{t} {s} {v}".split())
    for _ in range(2000):
        s = random.choice(subjects)
        v = random.choice(['found', 'held', 'carried', 'brought', 'took'])
        o = random.choice(objects)
        corpus.append(f"{s} {v} {o}".split())
    for _ in range(2000):
        s1, adj1 = random.choice(subjects), random.choice(adjectives)
        s2, adj2 = random.choice(['he', 'she', 'they']), random.choice(adjectives)
        corpus.append(f"{s1} was {adj1} and {s2} was {adj2}".split())
    for _ in range(2000):
        s, adj = random.choice(['the', 'a']), random.choice(adjectives)
        noun = random.choice(['man', 'woman', 'child', 'king', 'soldier'])
        v = random.choice(verbs)
        corpus.append(f"{s} {adj} {noun} {v}".split())

    random.shuffle(corpus)
    vocab = sorted(set(word for sent in corpus for word in sent))
    print(f"📚 Corpus: {len(corpus):,} sentences, {len(vocab)} words")
    return corpus, vocab

corpus, vocab = generate_rich_corpus()

# ===============================================================
# 4. YOUR TOPOLOGICAL NEURAL NETWORK (MHD + normalization fix)
# ===============================================================
class TopologicalNeuralNetwork(nn.Module):
    """
    YOUR TNN with MHD closure insights:
    - Uses RATIOS not differences (S_α = η·u/c)
    - Energy and coupling interact through division
    - Proper normalization like shared coordinate 'c'
    - E/C normalization fix to avoid scale blowup
    """
    def __init__(self, input_dim=256, hidden_dim=512, output_dim=256, lambda_topo=1.0):
        super().__init__()
        self.lambda_topo = lambda_topo
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.layer_norm = nn.LayerNorm(hidden_dim)
        self.energy = 0.0
        self.coupling = 0.0
        self.stability = 0.0
        self.energy_ratio = 1.0

    def compute_topological_terms(self, activations):
        """
        E = mean(h^2)
        dE/dh = (2/N) * h → mean((dE/dh)^2) = (4/N^2) * mean(h^2) = (4/N^2)*E
        Normalize C so C ≈ E in ideal equilibrium.
        """
        eps = 1e-8
        E = torch.mean(activations ** 2)

        grad = torch.autograd.grad(E, activations, retain_graph=True,
                                   create_graph=True, allow_unused=True)
        if grad[0] is None:
            C = E
        else:
            C_raw = torch.mean(grad[0] ** 2)
            N = activations.numel()
            # C_raw = (4/N^2) * E  ⇒  C = C_raw / (4/N^2) = C_raw * (N^2/4)
            scale = 4.0 / (N ** 2)
            C = C_raw / (scale + eps)

        ratio = E / (C + eps)
        ratio_clamped = torch.clamp(ratio, 0.25, 4.0)

        E_d = E.detach()
        C_d = C.detach()
        S_d = torch.abs(ratio - 1.0).detach()
        return E_d, C_d, S_d, ratio_clamped

    def forward(self, x):
        # Handle 3D input (batch, seq, dim)
        reshaped = False
        if len(x.shape) == 3:
            b, s, d = x.shape
            x = x.reshape(b * s, d)
            reshaped = True

        # Forward pass through layers
        h1 = F.relu(self.fc1(x))
        h2 = F.relu(self.fc2(h1))
        h2_norm = self.layer_norm(h2)
        out = self.fc3(h2_norm)

        # MHD-inspired correction using normalized ratio
        if self.training:
            E_d, C_d, S_d, ratio_for_ctrl = self.compute_topological_terms(h2_norm)
            self.energy, self.coupling, self.stability = E_d.item(), C_d.item(), S_d.item()

            # Smooth controller with EMA
            if not hasattr(self, "_ratio_ema"):
                self._ratio_ema = ratio_for_ctrl.detach()
            self._ratio_ema = 0.9 * self._ratio_ema + 0.1 * ratio_for_ctrl.detach()

            correction = self.lambda_topo * torch.log(self._ratio_ema + 1e-8)
            out = out + correction * torch.tanh(out)

            # For external logs, expose the unclamped ratio value (safe)
            self.energy_ratio = float((E_d / (C_d + 1e-8)).cpu())

        if reshaped:
            out = out.reshape(b, s, -1)
        return out

# ===============================================================
# 5. COMPLETE RDT MODEL (spatial loss with MHD insights)
# ===============================================================
class RDTLanguageModel(nn.Module):
    """
    COMPLETE MODEL WITH ALL YOUR COMPONENTS + MHD INSIGHTS:
    1. RDT shell structure
    2. YOUR Topological Neural Network (with MHD ratio correction)
    3. YOUR spatial index for embeddings
    4. Trained with YOUR Topological Adam
    5. MHD-inspired spatial loss (uses depth ratios)
    """
    def __init__(self, vocab, emb_dim=256, hidden_dim=512, alpha=1.5):
        super().__init__()
        self.alpha = alpha
        self.word2idx = {w:i for i,w in enumerate(vocab)}
        self.idx2word = {i:w for w,i in self.word2idx.items()}
        self.vocab_size = len(vocab)

        # COMPONENT 1: YOUR RDT SPATIAL INDEX
        self.rdt_index = RDTEmbeddingIndex(self.vocab_size, emb_dim, alpha)

        # COMPONENT 2: Embeddings
        self.embed = nn.Embedding(self.vocab_size, emb_dim)

        # COMPONENT 3: YOUR TOPOLOGICAL NEURAL NETWORK (with MHD)
        self.tnn = TopologicalNeuralNetwork(
            input_dim=emb_dim, hidden_dim=hidden_dim, output_dim=emb_dim, lambda_topo=1.0
        )

        # COMPONENT 4: Output layer
        self.fc_out = nn.Linear(emb_dim, self.vocab_size)

        print(f"\n📊 Complete Model:")
        print(f"   Parameters: {sum(p.numel() for p in self.parameters()):,}")
        print(f"   Components: RDT Index + TNN (MHD) + Spatial Search")

    def forward(self, x):
        # x: (batch, seq_len)
        e = self.embed(x)                # Embeddings
        h = self.tnn(e)                  # YOUR TNN with MHD correction
        out = self.fc_out(h)             # Output logits
        return out, h                    # Return hidden states for spatial loss

    def compute_rdt_spatial_loss(self, predictions, targets, hidden_states):
        """
        YOUR SPATIAL INDEX LOSS WITH MHD INSIGHTS:
        Uses RATIOS (like S_α = η·u/c) instead of products:
        - Shell depth ratio (like coordinate ratio in MHD)
        - Hierarchical search with proper scaling
        - Rewards structural similarity
        """
        batch_size, seq_len, vocab_size = predictions.shape
        pred_words = torch.argmax(predictions, dim=-1).view(-1)
        target_words = targets.view(-1)

        # Build embeddings dict for spatial search
        with torch.no_grad():
            embeddings_dict = {idx: self.embed.weight[idx] for idx in range(self.vocab_size)}

        total_bonus, n_valid = 0.0, 0

        # Sample for efficiency
        sample_size = min(100, len(pred_words))
        indices = random.sample(range(len(pred_words)), sample_size)

        for i in indices:
            pred_idx = pred_words[i].item()
            target_idx = target_words[i].item()
            if target_idx == 0:  # Skip padding
                continue

            hidden = hidden_states.view(-1, hidden_states.size(-1))[i]

            # Depth ratio bonus
            pred_depth = self.rdt_index.word_depth[pred_idx]
            target_depth = self.rdt_index.word_depth[target_idx]
            if pred_depth == target_depth:
                depth_bonus = 1.0
            else:
                min_depth = min(pred_depth, target_depth) + 1
                max_depth = max(pred_depth, target_depth) + 1
                depth_bonus = min_depth / max_depth  # ratio in (0,1]

            # Hierarchical spatial candidate check
            candidates = self.rdt_index.hierarchical_search(
                hidden, target_idx, embeddings_dict, k=10
            )
            if pred_idx in candidates:
                rank = candidates.index(pred_idx)
                rank_bonus = 1.0 / (rank + 1)
                combined_bonus = depth_bonus * rank_bonus
                total_bonus += combined_bonus
            else:
                total_bonus += depth_bonus * 0.1

            n_valid += 1

        return total_bonus / n_valid if n_valid > 0 else 0.0

# ===============================================================
# 6. TRAINING WITH YOUR TOPOLOGICAL ADAM
# ===============================================================
def prepare_batch(sentences, word2idx, maxlen=15):
    batch_x, batch_y = [], []
    for sent in sentences:
        if len(sent) < 2: continue
        idxs = [word2idx.get(w, 0) for w in sent]
        if len(idxs) < 2: continue
        x, y = idxs[:-1], idxs[1:]
        if len(x) < maxlen:
            x = x + [0] * (maxlen - len(x))
            y = y + [0] * (maxlen - len(y))
        else:
            x, y = x[:maxlen], y[:maxlen]
        batch_x.append(x)
        batch_y.append(y)
    if not batch_x:
        return None, None
    return torch.tensor(batch_x, device=device), torch.tensor(batch_y, device=device)

def train_model(model, corpus, epochs=50, initial_lr=1e-3, batch_size=64):
    optimizer = TopologicalAdam(model.parameters(), lr=initial_lr)
    criterion = nn.CrossEntropyLoss()
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=5
    )

    loss_log, spatial_log, energy_log, coupling_log, ratio_log, topo_energy_log = [], [], [], [], [], []
    best_loss = float('inf')

    print(f"\n🔥 Training COMPLETE SYSTEM WITH MHD INSIGHTS:")
    print(f"   ✓ RDT spatial index")
    print(f"   ✓ Topological Neural Network (MHD ratio correction)")
    print(f"   ✓ Topological Adam optimizer")
    print(f"   ✓ Hierarchical search with depth ratios")
    print(f"   ✓ MHD-inspired loss function\n")

    for ep in range(1, epochs+1):
        random.shuffle(corpus)
        epoch_loss = epoch_spatial = epoch_tnn_energy = epoch_tnn_coupling = epoch_ratio = 0.0
        n_batches = 0

        for i in range(0, len(corpus), batch_size):
            batch = corpus[i:i+batch_size]
            x, y = prepare_batch(batch, model.word2idx)
            if x is None: continue

            # Forward pass through COMPLETE system
            out, hidden = model(x)  # Embed → TNN (MHD) → Output

            # CE loss
            loss = criterion(out.view(-1, model.vocab_size), y.view(-1))

            # Spatial bonus
            spatial_bonus = model.compute_rdt_spatial_loss(out, y, hidden)

            # Combined objective (subtract bonus)
            spatial_weight = 0.5
            total_loss = loss - (spatial_weight * spatial_bonus)

            # Backprop with Topological Adam
            optimizer.zero_grad()
            total_loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            epoch_loss += loss.item()
            epoch_spatial += spatial_bonus
            epoch_tnn_energy += model.tnn.energy
            epoch_tnn_coupling += model.tnn.coupling
            epoch_ratio += model.tnn.energy_ratio
            n_batches += 1

        if n_batches:
            mean_loss = epoch_loss / n_batches
            mean_spatial = epoch_spatial / n_batches
            mean_tnn_energy = epoch_tnn_energy / n_batches
            mean_tnn_coupling = epoch_tnn_coupling / n_batches
            mean_ratio = epoch_ratio / n_batches
            mean_topo_energy = optimizer.energy()

            loss_log.append(mean_loss)
            spatial_log.append(mean_spatial)
            energy_log.append(mean_tnn_energy)
            coupling_log.append(mean_tnn_coupling)
            ratio_log.append(mean_ratio)
            topo_energy_log.append(mean_topo_energy)

            scheduler.step(mean_loss)
            improved = mean_loss < best_loss
            if improved: best_loss = mean_loss
            if ep % 5 == 0 or ep == 1:
                print(f"Epoch {ep:2d}: loss={mean_loss:.4f} | spatial={mean_spatial:.4f} | "
                      f"E/C_ratio={mean_ratio:.3f} | TNN_E={mean_tnn_energy:.2e} | "
                      f"Adam_E={mean_topo_energy:.2e} {'🔥' if improved else ''}")

    print(f"\n✅ Training Complete!")
    print(f"   Best loss: {best_loss:.4f}")
    print(f"   Final spatial bonus: {spatial_log[-1]:.4f}")
    print(f"   Final E/C ratio: {ratio_log[-1]:.3f}")

    return loss_log, spatial_log, energy_log, coupling_log, ratio_log, topo_energy_log
# ===============================================================
# 8. RUN COMPLETE SYSTEM
# ===============================================================
print("="*70)
print("COMPLETE SYSTEM: RDT + TNN (MHD) + TOPOLOGICAL ADAM + SPATIAL INDEX")
print("="*70)

torch.manual_seed(42)
model = RDTLanguageModel(vocab, emb_dim=256, hidden_dim=512, alpha=1.5).to(device)

losses, spatials, energies, couplings, ratios, adam_energies = train_model(
    model, corpus, epochs=50, initial_lr=1e-3, batch_size=64
)

print("\n" + "="*70)
print("TEXT GENERATION")
print("="*70)

for prompt in ["the man", "in the morning", "she was", "long ago"]:
    print(f"\n'{prompt}':")
    for i in range(3):
        print(f"  {generate_text(model, prompt, max_len=12)}")
# ===============================================================
# 7. GENERATION
# ===============================================================
def generate_text(model, start_text, max_len=15, temperature=0.8):
    model.eval()
    words = start_text.lower().split()
    with torch.no_grad():
        for _ in range(max_len):
            x = torch.tensor([[model.word2idx.get(w, 0) for w in words]], device=device)
            out, _ = model(x)
            logits = out[0, -1, :] / temperature
            probs = F.softmax(logits, dim=0)
            next_idx = torch.multinomial(probs, 1).item()
            if next_idx == 0: break
            words.append(model.idx2word[next_idx])
            if len(words) > 20: break
    return ' '.join(words)

# ===============================================================
# 8. RUN COMPLETE SYSTEM
# ===============================================================
print("="*70)
print("COMPLETE SYSTEM: RDT + TNN (MHD) + TOPOLOGICAL ADAM + SPATIAL INDEX")
print("="*70)

torch.manual_seed(42)
model = RDTLanguageModel(vocab, emb_dim=256, hidden_dim=512, alpha=1.5).to(device)

losses, spatials, energies, couplings, ratios, adam_energies = train_model(
    model, corpus, epochs=50, initial_lr=1e-3, batch_size=64
)

print("\n" + "="*70)
print("TEXT GENERATION")
print("="*70)

for prompt in ["the man", "in the morning", "

SyntaxError: unterminated string literal (detected at line 524) (ipython-input-3873109890.py, line 524)

In [ ]:
# ===============================================================
#  FIX PYTORCH / CUDA CONFLICT (COLAB SAFE)
# ===============================================================

# 1️⃣ Remove every preloaded Torch build
!pip uninstall -y torch torchvision torchaudio torchtext torchdata torchsummary

# 2️⃣ Fresh unified install (CUDA 12.6 build)
!pip install torch==2.8.0+cu126 torchvision==0.23.0+cu126 torchaudio==2.8.0+cu126 --index-url https://download.pytorch.org/whl/cu126
!pip install topological-adam --quiet

# 3️⃣ Verify everything before loading your model
import torch, torch.nn as nn, torch.nn.functional as F
from topological_adam import TopologicalAdam
print(f"🚀 Torch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")

Found existing installation: torch 2.8.0+cu126
Uninstalling torch-2.8.0+cu126:
  Successfully uninstalled torch-2.8.0+cu126
Found existing installation: torchvision 0.23.0+cu126
Uninstalling torchvision-0.23.0+cu126:
  Successfully uninstalled torchvision-0.23.0+cu126
Found existing installation: torchaudio 2.8.0+cu126
Uninstalling torchaudio-2.8.0+cu126:
  Successfully uninstalled torchaudio-2.8.0+cu126
Looking in indexes: https://download.pytorch.org/whl/cu126
  Using cached https://download.pytorch.org/whl/cu126/torch-2.8.0%2Bcu126-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached https://download.pytorch.org/whl/cu126/torchvision-0.23.0%2Bcu126-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (6.1 kB)
  Using cached https://download.pytorch.org/whl/cu126/torchaudio-2.8.0%2Bcu126-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (7.2 kB)
Using cached https://download.pytorch.org/whl/cu126/torch-2.8.0%2Bcu126-cp312-cp312-manylinux_2_28_x86_64.whl (821.8 MB)
Using cache

KeyboardInterrupt: 

In [ ]:


# ===============================================================
#  COMPLETE RDT LANGUAGE MODEL - FIXED IMPORT ORDER
# ===============================================================

# 1️⃣  Restart-safe reinstall
!pip install torch==2.3.1 topological-adam --quiet --upgrade

# 2️⃣  Import AFTER install (avoid double TORCH_LIBRARY registration)
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np, math, random
from collections import defaultdict
from topological_adam import TopologicalAdam
from matplotlib import pyplot as plt
from numba import njit

# ===============================================================
# 1. RDT MATH
# ===============================================================
def rdt_depth(n, alpha=1.5):
    if n < 2: return 0
    x, depth = n, 0
    while x > 1 and depth < 1000:
        d = max(2, int(math.log(x) ** alpha))
        if x < d: break
        x //= d
        depth += 1
    return depth

@njit(fastmath=True)
def rdt_grid_size(n, alpha=1.5):
    if n <= 1:
        return 2
    d = max(2, int(np.log(n + 1) ** alpha))
    return min(d, 32)

# ===============================================================
# 2. RDT SPATIAL INDEX
# ===============================================================
class RDTEmbeddingIndex:
    def __init__(self, vocab_size, embedding_dim, alpha=1.5):
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.alpha = alpha
        self.shells = defaultdict(list)
        self.word_depth = {}
        for idx in range(vocab_size):
            depth = rdt_depth(idx, alpha)
            self.shells[depth].append(idx)
            self.word_depth[idx] = depth
        self.shell_grid_sizes = {}
        for depth, words in self.shells.items():
            n = len(words)
            self.shell_grid_sizes[depth] = rdt_grid_size(n, alpha)
        print(f"\n🔬 RDT Spatial Index:")
        for depth in sorted(self.shells.keys()):
            n = len(self.shells[depth])
            g = self.shell_grid_sizes[depth]
            print(f"   Shell {depth}: {n} words, grid {g}×{g}")

    def find_nearest_in_shell(self, embedding, target_shell, embeddings_dict, k=5):
        if target_shell not in self.shells:
            return []
        candidates = self.shells[target_shell]
        distances = []
        for word_idx in candidates:
            emb = embeddings_dict[word_idx]
            sim = F.cosine_similarity(embedding.unsqueeze(0), emb.unsqueeze(0))
            distances.append((word_idx, sim.item()))
        distances.sort(key=lambda x: x[1], reverse=True)
        return [idx for idx, _ in distances[:k]]

    def hierarchical_search(self, embedding, current_word_idx, embeddings_dict, k=10):
        current_shell = self.word_depth.get(current_word_idx, 0)
        candidates = self.find_nearest_in_shell(embedding, current_shell, embeddings_dict, k=k)
        if len(candidates) < k:
            for offset in [1, -1, 2, -2]:
                nearby_shell = current_shell + offset
                if nearby_shell in self.shells:
                    more = self.find_nearest_in_shell(embedding, nearby_shell, embeddings_dict, k=k-len(candidates))
                    candidates.extend(more)
                    if len(candidates) >= k:
                        break
        return candidates[:k]

# ===============================================================
# 3. CORPUS
# ===============================================================
def generate_rich_corpus():
    subjects = [
        'the man', 'the woman', 'the child', 'the king', 'the queen',
        'the soldier', 'the merchant', 'the teacher', 'the old man',
        'the young woman', 'a stranger', 'the traveler', 'the farmer',
        'the priest', 'the warrior', 'the beggar', 'the noble'
    ]
    verbs = [
        'walked','ran','went','came','spoke','thought','found','discovered',
        'remembered','believed','understood','left','arrived','departed',
        'searched','wondered','feared','hoped','carried','held','took','gave','brought','sent'
    ]
    locations = [
        'to the house','to the market','to the river','to the forest',
        'to the castle','through the valley','across the land','to the mountain',
        'to the sea','to the village','to the city','through the door',
        'into the darkness','toward the light','beyond the horizon','near the gate'
    ]
    adjectives = [
        'happy','sad','brave','wise','afraid','calm','strong','gentle','proud','humble',
        'kind','cruel','young','old','rich','poor','noble','simple','great','small'
    ]
    times = [
        'in the morning','at night','at dawn','at dusk','yesterday','long ago',
        'one day','when winter came','when spring arrived','at midnight',
        'before sunrise','after sunset'
    ]
    objects = [
        'a sword','a book','a letter','a key','a treasure','a secret','a truth',
        'a dream','a memory','a promise','a gift'
    ]
    corpus=[]
    for _ in range(4000):
        corpus.append(f"{random.choice(subjects)} {random.choice(verbs)} {random.choice(locations)}".split())
    for _ in range(3000):
        corpus.append(f"{random.choice(subjects)} was {random.choice(adjectives)}".split())
    for _ in range(3000):
        corpus.append(f"{random.choice(times)} {random.choice(subjects)} {random.choice(verbs)}".split())
    for _ in range(2000):
        corpus.append(f"{random.choice(subjects)} {random.choice(['found','held','carried','brought','took'])} {random.choice(objects)}".split())
    for _ in range(2000):
        s1, adj1 = random.choice(subjects), random.choice(adjectives)
        s2, adj2 = random.choice(['he','she','they']), random.choice(adjectives)
        corpus.append(f"{s1} was {adj1} and {s2} was {adj2}".split())
    for _ in range(2000):
        corpus.append(f"{random.choice(['the','a'])} {random.choice(adjectives)} {random.choice(['man','woman','child','king','soldier'])} {random.choice(verbs)}".split())
    random.shuffle(corpus)
    vocab = sorted(set(word for sent in corpus for word in sent))
    print(f"📚 Corpus: {len(corpus):,} sentences, {len(vocab)} words")
    return corpus, vocab

corpus,vocab = generate_rich_corpus()

# ===============================================================
# 4. TOPOLOGICAL NEURAL NETWORK
# ===============================================================
class TopologicalNeuralNetwork(nn.Module):
    def __init__(self, input_dim=256, hidden_dim=512, output_dim=256, lambda_topo=1.0):
        super().__init__()
        self.lambda_topo=lambda_topo
        self.fc1=nn.Linear(input_dim,hidden_dim)
        self.fc2=nn.Linear(hidden_dim,hidden_dim)
        self.fc3=nn.Linear(hidden_dim,output_dim)
        self.layer_norm=nn.LayerNorm(hidden_dim)
        self.energy=self.coupling=self.stability=self.energy_ratio=0.0

    def compute_topological_terms(self,activations):
        eps=1e-8
        E=torch.mean(activations**2)
        grad=torch.autograd.grad(E,activations,retain_graph=True,create_graph=True,allow_unused=True)
        if grad[0] is None:
            C=E
        else:
            C_raw=torch.mean(grad[0]**2)
            N=activations.numel()
            scale=4.0/(N**2)
            C=C_raw/(scale+eps)
        ratio=E/(C+eps)
        ratio_clamped=torch.clamp(ratio,0.25,4.0)
        E_d,C_d,S_d=E.detach(),C.detach(),torch.abs(ratio-1.0).detach()
        return E_d,C_d,S_d,ratio_clamped

    def forward(self,x):
        reshaped=False
        if len(x.shape)==3:
            b,s,d=x.shape
            x=x.reshape(b*s,d)
            reshaped=True
        h1=F.relu(self.fc1(x))
        h2=F.relu(self.fc2(h1))
        h2n=self.layer_norm(h2)
        out=self.fc3(h2n)
        if self.training:
            E_d,C_d,S_d,ratio_ctrl=self.compute_topological_terms(h2n)
            self.energy,self.coupling,self.stability=E_d.item(),C_d.item(),S_d.item()
            if not hasattr(self,"_ratio_ema"): self._ratio_ema=ratio_ctrl.detach()
            self._ratio_ema=0.9*self._ratio_ema+0.1*ratio_ctrl.detach()
            correction=self.lambda_topo*torch.log(self._ratio_ema+1e-8)
            out=out+correction*torch.tanh(out)
            self.energy_ratio=float((E_d/(C_d+1e-8)).cpu())
        if reshaped: out=out.reshape(b,s,-1)
        return out

# ===============================================================
# 5. COMPLETE MODEL
# ===============================================================
class RDTLanguageModel(nn.Module):
    def __init__(self,vocab,emb_dim=256,hidden_dim=512,alpha=1.5):
        super().__init__()
        self.alpha=alpha
        self.word2idx={w:i for i,w in enumerate(vocab)}
        self.idx2word={i:w for w,i in self.word2idx.items()}
        self.vocab_size=len(vocab)
        self.rdt_index=RDTEmbeddingIndex(self.vocab_size,emb_dim,alpha)
        self.embed=nn.Embedding(self.vocab_size,emb_dim)
        self.tnn=TopologicalNeuralNetwork(input_dim=emb_dim,hidden_dim=hidden_dim,output_dim=emb_dim)
        self.fc_out=nn.Linear(emb_dim,self.vocab_size)
        print(f"\n📊 Complete Model:\n   Parameters: {sum(p.numel() for p in self.parameters()):,}\n   Components: RDT Index + TNN (MHD) + Spatial Search")

    def forward(self,x):
        e=self.embed(x)
        h=self.tnn(e)
        out=self.fc_out(h)
        return out,h

    def compute_rdt_spatial_loss(self,predictions,targets,hidden_states):
        b,s,v=predictions.shape
        preds=torch.argmax(predictions,dim=-1).view(-1)
        targs=targets.view(-1)
        with torch.no_grad():
            emb_dict={i:self.embed.weight[i] for i in range(self.vocab_size)}
        total,n=0.0,0
        for i in random.sample(range(len(preds)),min(100,len(preds))):
            p,t=preds[i].item(),targs[i].item()
            if t==0: continue
            h=hidden_states.view(-1,hidden_states.size(-1))[i]
            pd,td=self.rdt_index.word_depth[p],self.rdt_index.word_depth[t]
            depth_bonus=1.0 if pd==td else (min(pd,td)+1)/(max(pd,td)+1)
            cands=self.rdt_index.hierarchical_search(h,t,emb_dict,k=10)
            if p in cands:
                r=cands.index(p)
                total+=depth_bonus/(r+1)
            else: total+=depth_bonus*0.1
            n+=1
        return total/n if n>0 else 0.0

# ===============================================================
# 6. TRAINING
# ===============================================================
def prepare_batch(sents,word2idx,maxlen=15):
    bx,by=[],[]
    for s in sents:
        if len(s)<2: continue
        idxs=[word2idx.get(w,0) for w in s]
        x,y=idxs[:-1],idxs[1:]
        if len(x)<maxlen:
            x=x+[0]*(maxlen-len(x));y=y+[0]*(maxlen-len(y))
        else: x,y=x[:maxlen],y[:maxlen]
        bx.append(x);by.append(y)
    if not bx: return None,None
    return torch.tensor(bx,device=device),torch.tensor(by,device=device)

def train_model(model,corpus,epochs=50,initial_lr=1e-3,batch_size=64):
    opt=TopologicalAdam(model.parameters(),lr=initial_lr)
    crit=nn.CrossEntropyLoss()
    sch=torch.optim.lr_scheduler.ReduceLROnPlateau(opt,mode='min',factor=0.5,patience=5)
    L,S,E,C,R,AE=[],[],[],[],[],[]
    best=float('inf')
    print("\n🔥 Training COMPLETE SYSTEM WITH MHD INSIGHTS:")
    for ep in range(1,epochs+1):
        random.shuffle(corpus)
        el,es,ee,ec,er,nb=0,0,0,0,0,0
        for i in range(0,len(corpus),batch_size):
            b=corpus[i:i+batch_size]
            x,y=prepare_batch(b,model.word2idx)
            if x is None: continue
            o,h=model(x)
            loss=crit(o.view(-1,model.vocab_size),y.view(-1))
            sb=model.compute_rdt_spatial_loss(o,y,h)
            tl=loss-0.5*sb
            opt.zero_grad();tl.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(),1.0)
            opt.step()
            el+=loss.item();es+=sb;ee+=model.tnn.energy;ec+=model.tnn.coupling;er+=model.tnn.energy_ratio;nb+=1
        if nb:
            ml,ms,me,mc,mr,ma=el/nb,es/nb,ee/nb,ec/nb,er/nb,opt.energy()
            L.append(ml);S.append(ms);E.append(me);C.append(mc);R.append(mr);AE.append(ma)
            sch.step(ml)
            imp=ml<best
            if imp: best=ml
            if ep%5==0 or ep==1:
                print(f"Epoch {ep:2d}: loss={ml:.4f} | spatial={ms:.4f} | E/C_ratio={mr:.3f} | TNN_E={me:.2e} | Adam_E={ma:.2e} {'🔥' if imp else ''}")
    print(f"\n✅ Done! Best loss {best:.4f}")
    return L,S,E,C,R,AE

# ===============================================================
# 7. GENERATION
# ===============================================================
def generate_text(model,start_text,max_len=15,temperature=0.8):
    model.eval();w=start_text.lower().split()
    with torch.no_grad():
        for _ in range(max_len):
            x=torch.tensor([[model.word2idx.get(t,0) for t in w]],device=device)
            o,_=model(x)
            logit=o[0,-1,:]/temperature
            probs=F.softmax(logit,dim=0)
            n=torch.multinomial(probs,1).item()
            if n==0: break
            w.append(model.idx2word[n])
            if len(w)>20: break
    return ' '.join(w)

# ===============================================================
# 8. RUN SYSTEM
# ===============================================================
print("="*70)
print("COMPLETE SYSTEM: RDT + TNN (MHD) + TOPOLOGICAL ADAM + SPATIAL INDEX")
print("="*70)
torch.manual_seed(42)
model=RDTLanguageModel(vocab,emb_dim=256,hidden_dim=512,alpha=1.5).to(device)
L,S,E,C,R,AE=train_model(model,corpus,epochs=50,initial_lr=1e-3,batch_size=64)

print("\n"+"="*70)
print("TEXT GENERATION")
print("="*70)
for p in ["the man","in the morning","she was","long ago"]:
    print(f"\n'{p}':")
    for i in range(3):
        print(f"  {generate_text(model,p,max_len=12)}")

# ===============================================================
# 9. VISUALIZATION
# ===============================================================
fig,ax=plt.subplots(2,3,figsize=(18,10))
ax[0,0].plot(L,'b-',lw=2);ax[0,0].axhline(y=0.5,color='r',ls='--',alpha=0.5);ax[0,0].set_title("Loss");ax[0,0].grid(True)
ax[0,1].plot(S,'g-',lw=2);ax[0,1].set_title("RDT Spatial Bonus");ax[0,1].grid(True)
ax[0,2].plot(R,'orange',lw=2);ax[0,2].axhline(y=1.0,color='r',ls='--',alpha=0.5);ax[0,2].set_title("E/C Ratio");ax[0,2].grid(True)
ax[1,0].plot(E,'r-',lw=2,label='Energy');ax[1,0].plot(C,'purple',lw=2,label='Coupling');ax[1,0].legend();ax[1,0].set_title("TNN Energy & Coupling");ax[1,0].grid(True)
ax[1,1].plot(AE,'darkblue',lw=2);ax[1,1].set_title("Topological Adam Energy");ax[1,1].grid(True)
ax[1,2].plot(L,'b-',lw=2,alpha=0.7,label='Loss');ax[1,2].plot([s*2 for s in S],'g-',lw=2,alpha=0.7,label='Spatial×2');ax[1,2].legend();ax[1,2].set_title("Loss vs Spatial");ax[1,2].grid(True)
plt.tight_layout();plt.show()

print("\n" + "="*70)
print("FINAL METRICS")
print("="*70)
print(f"Loss: {L[-1]:.4f}")
print(f"Spatial bonus: {S[-1]:.4f}")
print(f"E/C ratio: {R[-1]:.3f} (ideal=1.0)")
print(f"TNN energy: {E[-1]:.2e}")
print(f"TNN coupling: {C[-1]:.2e}")
print(f"Topological Adam energy: {AE[-1]:.2e}")
print("="*70)
print("✅ ALL COMPONENTS ACTIVE & STABLE (MHD normalized)")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 6.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not cu

AttributeError: type object 'GuardSource' has no attribute 'LOCAL_NN_MODULE'

In [ ]:
# ===============================================================
#  CLEAN TORCH + TOPOLOGICAL ADAM INSTALL (COMPATIBLE VERSIONS)
# ===============================================================

!pip install torch==2.8.0 torchvision==0.23.0 torchaudio==2.8.0 --quiet --extra-index-url https://download.pytorch.org/whl/cu126
!pip install topological-adam --quiet

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np, math, random
from collections import defaultdict
from topological_adam import TopologicalAdam
from matplotlib import pyplot as plt
from numba import njit

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Device: {device}")
print(f"✅ Torch version: {torch.__version__}")

🚀 Device: cpu
✅ Torch version: 2.8.0+cu126


In [ ]:
# ============================================================
# 🔮 Topological State Manager v1 — RDT + TNN Integrated Memory
# ============================================================

import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import defaultdict

class TopologicalStateManager(nn.Module):
    """
    Field-based persistent memory for the Topological-Geometric LM.
    Stores and retrieves energy configurations (Φ) across sessions.
    Integrates directly with the TNN and RDT geometry.
    """

    def __init__(self, dim, decay=0.995, coupling=0.75, device=None):
        super().__init__()
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        self.dim = dim
        self.decay = decay        # energy decay factor per update
        self.coupling = coupling  # field coupling coefficient κ
        self.memory = defaultdict(lambda: torch.zeros(dim, device=self.device))
        self.energy_map = defaultdict(float)
        self.step = 0

    # --------------------------------------------
    # Encode: store current field into memory
    # --------------------------------------------
    def encode(self, shell_id, field_vec, energy):
        if field_vec is None: return
        if shell_id not in self.memory:
            self.memory[shell_id] = field_vec.detach().clone()
            self.energy_map[shell_id] = energy
        else:
            old = self.memory[shell_id]
            # weighted merge by energy + coupling
            merged = (
                self.coupling * field_vec +
                (1 - self.coupling) * old * self.decay
            )
            self.memory[shell_id] = merged.detach().clone()
            self.energy_map[shell_id] = (
                self.coupling * energy + (1 - self.coupling) * self.energy_map[shell_id]
            )
        self.step += 1

    # --------------------------------------------
    # Recall: retrieve a field state near a shell
    # --------------------------------------------
    def recall(self, shell_id, radius=1):
        # get nearest memory shell by geometric distance
        keys = list(self.memory.keys())
        if not keys:
            return torch.zeros(self.dim, device=self.device), 0.0
        diffs = [abs(k - shell_id) for k in keys]
        nearest = keys[int(torch.argmin(torch.tensor(diffs)))]
        return self.memory[nearest], self.energy_map[nearest]

    # --------------------------------------------
    # Global Decay: periodic relaxation to equilibrium
    # --------------------------------------------
    def relax(self):
        for k in list(self.memory.keys()):
            self.memory[k] *= self.decay
            self.energy_map[k] *= self.decay

    # --------------------------------------------
    # Entropy Estimate (diagnostic)
    # --------------------------------------------
    def entropy(self):
        vals = torch.stack(list(self.memory.values())) if self.memory else torch.zeros(1, self.dim, device=self.device)
        return torch.mean(torch.var(vals, dim=0)).item()

# ============================================================
# 🧬 Integration Hook for your LM
# ============================================================

def integrate_state_manager(model, dim):
    """
    Attaches the TSM to an existing model instance.
    """
    model.state_manager = TopologicalStateManager(dim=dim, device=next(model.parameters()).device)

    def after_forward_hook(module, input, output):
        # automatically store field after each batch
        if hasattr(module, 'depth'):
            energy = getattr(module, 'field_energy', 1.0)
            model.state_manager.encode(shell_id=module.depth, field_vec=output.mean(0), energy=energy)

    # hook into your TNN forward output
    for name, module in model.named_modules():
        if isinstance(module, nn.Linear):
            module.register_forward_hook(after_forward_hook)

    print("✅ Topological State Manager integrated with model.")
    return model


# ============================================================
# ✅ Example Integration (Run Once)
# ============================================================

# assuming your model variable is `model`
model = integrate_state_manager(model, dim=256)

# Example usage:
# During training, states auto-encode via hooks.
# You can manually recall and inject context like this:

shell = 3
field_vec, energy = model.state_manager.recall(shell)
print(f"🔁 Recalled shell {shell} | Energy={energy:.4f} | Entropy≈{model.state_manager.entropy():.6f}")

NameError: name 'model' is not defined